In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test_label.csv
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test.data
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/.DS_Store
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test.map
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train_data.csv
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train_label.csv
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/vocabulary.txt
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.label
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test.label
/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test_data.csv
/Users/yongxingnie/Desk

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Calculate class distribution (prior probability)
train_label=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.label')
df_train_label=pd.read_csv(train_label,names=['classId'])
df_train_label.head()

,classId
0,1
1,1
2,1
3,1
4,1


In [4]:
#Calculate logPrior
def Prior(df_label):
    
    prior=(df_label.groupby(['classId']).apply(lambda x:len(x))/len(df_label))
    
    return prior

In [5]:
#Getting the logPrior for the train label data
prior=Prior(df_train_label)
prior

classId
1     0.042595
2     0.051557
3     0.050759
4     0.052090
5     0.051025
6     0.052533
7     0.051646
8     0.052533
9     0.052888
10    0.052711
11    0.053066
12    0.052711
13    0.052445
14    0.052711
15    0.052622
16    0.053155
17    0.048363
18    0.050049
19    0.041175
20    0.033366
dtype: float64

In [6]:
#Common stop words from online
stop_words = [
"a", "about", "above", "across", "after", "afterwards", 
"again", "all", "almost", "alone", "along", "already", "also",    
"although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "as", "at", "be", "became", "because", "become","becomes", "becoming", "been", "before", "behind", "being", "beside", "besides", "between", "beyond", "both", "but", "by","can", "cannot", "cant", "could", "couldnt", "de", "describe", "do", "done", "each", "eg", "either", "else", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "find","for","found", "four", "from", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "indeed", "is", "it", "its", "itself", "keep", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next","no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part","perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "she", "should","since", "sincere","so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "take","than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they",
"this", "those", "though", "through", "throughout",
"thru", "thus", "to", "together", "too", "toward", "towards",
"under", "until", "up", "upon", "us",
"very", "was", "we", "well", "were", "what", "whatever", "when",
"whence", "whenever", "where", "whereafter", "whereas", "whereby",
"wherein", "whereupon", "wherever", "whether", "which", "while", 
"who", "whoever", "whom", "whose", "why", "will", "with",
"within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
]

In [7]:
#Creating an vocabulary dataframe
vocab = open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/vocabulary.txt') 
vocab_df = pd.read_csv(vocab, names = ['word']) 
vocab_df = vocab_df.reset_index() 
vocab_df['index'] = vocab_df['index'].apply(lambda x: x+1) 
vocab_df.head()

,index,word
0,1,archive
1,2,name
2,3,atheism
3,4,resources
4,5,alt


In [8]:
#Getting the count of each word in the vocabulary
#Index of all words
tot_list = set(vocab_df['index'])

#Index of good words
vocab_df = vocab_df[~vocab_df['word'].isin(stop_words)]
good_list = vocab_df['index'].tolist()
good_list = set(good_list)

#Index of stop words
bad_list = tot_list - good_list

len(bad_list)

250

In [9]:
def mlikelihood(df_train, df_label, badlist):
    #1. consolidating the train data and the train label
    #df_train should be in format of dataframe
    #df_label should have a format of list
    
    a=0.001
    
    label=[]
    lines=df_label.readlines()
    for line in lines:
        label.append(int(line.split()[0]))
    
    #increasing label length to match docId
    docId=df_train['docId'].values
    i=0
    new_label=[]
    for index in range(len(docId)-1):
        new_label.append(label[i])
        if docId[index]!=docId[index+1]:
            i+=1
    new_label.append(label[i])

    df_train['classId']=new_label
    
    #2. Calculating the probablity of each term given a classId
    pb_ij=df_train.groupby(['classId','termId'])
    pb_j=df_train.groupby(['classId'])
    
    pr=((pb_ij['count'].sum()+a)/(pb_j['count'].sum()+16689))
    pr=pr.unstack()
    
    #3. Smoothing the data and replace the NaN with a/(count+|V|+1)
    # V is the number of terms in the vocabulary
    for c in range(1,21):
        pr.loc[c,:]=pr.loc[c,:].fillna(a/(pb_j['count'].sum()[c]+16689))
   
    pr_dict=pr.to_dict()
    
    for bad in badlist:
        for j in range(1,21):
            pr_dict[j][bad]=0
    
    return pr_dict

In [10]:
train_label=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.label')
train_data=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.data')
train_data=pd.read_csv(train_data, delimiter=' ',names=['docId','termId','count'])
pr_dict=mlikelihood(train_data, train_label, bad_list)
pr_dict

{1: {1: 7.855541658358559e-05,
  2: 0,
  3: 0.00010237678677784395,
  4: 6.907238744766262e-05,
  5: 5.833066029024388e-05,
  6: 0.00027723478927655054,
  7: 1.2856279649794942e-08,
  8: 6.881972001131575e-05,
  9: 0.00011733992624874288,
  10: 8.034546140448039e-06,
  11: 6.337207830028615e-06,
  12: 0,
  13: 2.5037126028265837e-05,
  14: 8.720142768286373e-05,
  15: 0.0002816910500401988,
  16: 0,
  17: 9.86537073669673e-05,
  18: 3.683690983962813e-05,
  19: 4.923319301873323e-09,
  20: 7.364583716905402e-09,
  1025: 0,
  2562: 0,
  1028: 0,
  1033: 0,
  1034: 0,
  1545: 0,
  1036: 0,
  1547: 0,
  527: 0,
  1040: 0,
  1041: 0,
  531: 0,
  23: 0,
  536: 0,
  25: 0,
  38938: 0,
  27: 0,
  29: 0,
  30: 0,
  31: 0,
  2592: 0,
  33: 0,
  2081: 0,
  38951: 0,
  2600: 0,
  1065: 0,
  42: 0,
  2090: 0,
  48: 0,
  49: 0,
  4145: 0,
  51: 0,
  52: 0,
  563: 0,
  1077: 0,
  1079: 0,
  7218: 0,
  1595: 0,
  60: 0,
  575: 0,
  15937: 0,
  578: 0,
  2114: 0,
  72: 0,
  73: 0,
  584: 0,
  1099: 0,

In [11]:
#Doing the naive bayes classifier

In [12]:
def naiveBayesClassify(prior, mlikelihood, testData):
    #testdata should have a format of dataframe with cloumn names ['docId', 'termId', 'count']
    #logprior is the result of logPrior function
    #loglikelihood is the result of mloglikelihood function
    
    tot=len(testData['docId'].unique())
    pb_ij=testData.groupby(['termId'])
    IDF=np.log(tot/pb_ij['docId'].count())
    IDF_dict=IDF.to_dict()
    
    testData_dict=testData.to_dict()
    new_dict={}
    prediction=[]
    
    #transforming the testData into the new_dict with a form as {{docId:{termId:count},{termId:count},..}...}}
    for idx in range(len(testData_dict['docId'])):
        docIdx=testData_dict['docId'][idx]
        termIdx=testData_dict['termId'][idx]
        count=testData_dict['count'][idx]
        
        new_dict[testData_dict['docId'][idx]] = {}
        new_dict[docIdx][termIdx]=count
    
    #Calculating the probability for each doc
    for docIdx in range(1, len(new_dict)+1):
        prob_dict={}
        
        #1. Creating a probability row 'prob_dict' to collect the probability of each class
        # given the different terms in a doc in the testData
        for classIdx in range(1,21):
            prob_dict[classIdx]=0
            
            #2. From the loglikelihood, retrieving the term probability in a give class
            for termIdx in new_dict[docIdx]:
                try:
                    probability=mlikelihood[termIdx][classIdx]
                    power=np.log(1+new_dict[docIdx][termIdx])
                    prob_dict[classIdx]+=power*np.log(probability*IDF_dict[termIdx])
                except:
                    #Missing V will have log(1+0)*log(a/16689)=0
                    prob_dict[classIdx]+=0
                
                 #calculating a class probability given the appeared terms
            #Adding the posterior of given class
            prob_dict[classIdx]+=np.log(prior[classIdx])
        
        #Getting the class with the max Probability for a given doc
        max_score=max(prob_dict, key=prob_dict.get)
        prediction.append(max_score)
    
    return prediction
        

In [13]:
#calculating the prediciton and comparing it with the test label

In [14]:
test_data=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test.data')
test_data=pd.read_csv(test_data, delimiter=' ',names=['docId','termId','count'])
pred=naiveBayesClassify(prior, pr_dict, test_data)

In [15]:
pred

[11,
 1,
 20,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 16,
 16,
 16,
 18,
 3,
 16,
 16,
 7,
 16,
 4,
 8,
 15,
 16,
 16,
 20,
 16,
 16,
 16,
 18,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 18,
 16,
 15,
 16,
 6,
 10,
 16,
 16,
 16,
 16,
 16,
 17,
 16,
 17,
 16,
 20,
 16,
 16,
 15,
 16,
 9,
 16,
 14,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 12,
 15,
 15,
 15,
 16,
 9,
 12,
 20,
 16,
 16,
 14,
 6,
 16,
 3,
 13,
 15,
 16,
 16,
 16,
 16,
 15,
 16,
 5,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 6,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 19,
 16,
 13,
 19,
 19,
 16,
 16,
 16,
 13,
 16,
 16,
 16,
 16,
 15,
 16,
 16,
 16,
 20,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 16,
 19,
 16,
 16,
 16,
 20,
 16,
 16,
 20,
 16,
 16,
 16,
 16,
 11,
 11,
 16,
 18,
 15,
 17,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 1,
 16,
 16,
 20,
 16,
 16,
 19,
 19,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 20,
 20,
 16,
 15,
 16,
 16,
 16,
 16,
 16,
 1,
 16,
 16,
 16,
 16,
 16,
 16,

In [16]:
#calculate the prediction accuracy

In [17]:
#getting the test label
test_label=pd.read_csv('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test_label.csv', names=['t'])
test_label=test_label['t'].tolist()

total=len(test_label)

value=0
for i,j in zip(pred, test_label):
    if i==j:
        value+=1
    else:
        pass

print("Accuracy:\t", (value/total)*100, "%")

Accuracy:	 17.948034643570953 %


In [18]:
#since the terms is removed around 0.5%, we need to increase the percentage.

In [19]:
#first of all, we need to calculate the frequency of all the terms in the train data

In [20]:
train_label_2=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.label')
train_data_2=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.data')
train_data_2=pd.read_csv(train_data_2, delimiter=' ',names=['docId','termId','count'])

In [21]:
pb_j=train_data_2.groupby(['termId'], sort=False)['count'].sum().sort_values(ascending=False)
print (pb_j.tail(38975))

termId
8007     11
6965     11
577      11
35632    11
47076    11
         ..
48021     1
48020     1
48019     1
42843     1
53975     1
Name: count, Length: 38975, dtype: int64


In [22]:
#I find that most of the words appear only once, and there only 15000 (27.8%) words appears 10 times or above.
#Then I will set 10 times as a benchmark, words whose occurrence are lower than 10 will be labeled as bad words.


In [23]:
badlist_2=pb_j.tail(38975).to_frame()

In [24]:
badlist_2

,count
termId,
8007,11
6965,11
577,11
35632,11
47076,11
...,...
48021,1
48020,1
48019,1


In [25]:
badlist_2=badlist_2.index.values.tolist()
badlist_2

[8007,
 6965,
 577,
 35632,
 47076,
 33707,
 23514,
 33711,
 26133,
 33717,
 47031,
 13944,
 35743,
 19296,
 46927,
 23413,
 33351,
 33352,
 52160,
 19433,
 49523,
 19423,
 14488,
 23239,
 26476,
 35980,
 2810,
 1431,
 19396,
 8207,
 17849,
 19386,
 33409,
 35965,
 24310,
 35787,
 2832,
 47317,
 24324,
 6904,
 49526,
 8140,
 26320,
 24326,
 35762,
 33728,
 24960,
 6987,
 24134,
 19128,
 35592,
 33797,
 18131,
 26073,
 11802,
 33855,
 24388,
 7014,
 23626,
 33913,
 33895,
 7869,
 18097,
 11274,
 26100,
 7884,
 23563,
 33866,
 24145,
 23586,
 33785,
 7910,
 1459,
 14388,
 35626,
 33762,
 35574,
 24380,
 46347,
 33773,
 46869,
 12197,
 34008,
 1699,
 13960,
 11301,
 23555,
 25861,
 28304,
 15873,
 24289,
 24807,
 24128,
 28900,
 44204,
 28595,
 28300,
 53674,
 24127,
 24311,
 24898,
 48289,
 28575,
 24308,
 23995,
 53697,
 24306,
 15758,
 44232,
 40348,
 49803,
 24331,
 16447,
 21799,
 24233,
 53171,
 24584,
 24510,
 40185,
 28797,
 24562,
 24091,
 42319,
 49792,
 21990,
 44307,
 24074,
 

In [26]:
#Then apply the badlist to the MNB again

In [27]:
train_label_3=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.label')
train_data_3=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/train.data')
train_data_3=pd.read_csv(train_data_3, delimiter=' ',names=['docId','termId','count'])
pr_dict_3=mlikelihood(train_data_3, train_label_3, badlist_2)
pr_dict_3

{1: {1: 7.855541658358559e-05,
  2: 0.00047227404031578864,
  3: 0.00010237678677784395,
  4: 6.907238744766262e-05,
  5: 5.833066029024388e-05,
  6: 0.00027723478927655054,
  7: 1.2856279649794942e-08,
  8: 6.881972001131575e-05,
  9: 0.00011733992624874288,
  10: 8.034546140448039e-06,
  11: 6.337207830028615e-06,
  12: 0.00023947592622441225,
  13: 2.5037126028265837e-05,
  14: 8.720142768286373e-05,
  15: 0.0002816910500401988,
  16: 4.588081998201472e-09,
  17: 9.86537073669673e-05,
  18: 3.683690983962813e-05,
  19: 4.923319301873323e-09,
  20: 7.364583716905402e-09,
  8007: 0,
  6965: 0,
  577: 0,
  35632: 0,
  47076: 0,
  33707: 0,
  23514: 0,
  33711: 0,
  26133: 0,
  33717: 0,
  47031: 0,
  13944: 0,
  35743: 0,
  19296: 0,
  46927: 0,
  23413: 0,
  33351: 0,
  33352: 0,
  52160: 0,
  19433: 0,
  49523: 0,
  19423: 0,
  14488: 0,
  23239: 0,
  26476: 0,
  35980: 0,
  2810: 0,
  1431: 0,
  19396: 0,
  8207: 0,
  17849: 0,
  19386: 0,
  33409: 0,
  35965: 0,
  24310: 0,
  35787

In [28]:
test_data_2=open('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test.data')
test_data_2=pd.read_csv(test_data_2, delimiter=' ',names=['docId','termId','count'])
pred_2=naiveBayesClassify(prior, pr_dict_3, test_data_2)

In [29]:
pred_2

[11,
 1,
 20,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 16,
 16,
 16,
 18,
 3,
 16,
 16,
 7,
 16,
 4,
 8,
 15,
 16,
 16,
 20,
 16,
 16,
 16,
 18,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 18,
 16,
 15,
 16,
 6,
 10,
 16,
 16,
 16,
 16,
 16,
 17,
 16,
 17,
 16,
 20,
 16,
 16,
 15,
 16,
 9,
 16,
 14,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 12,
 15,
 15,
 15,
 16,
 9,
 12,
 20,
 16,
 16,
 14,
 6,
 16,
 3,
 13,
 15,
 16,
 16,
 16,
 16,
 15,
 16,
 5,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 6,
 16,
 16,
 16,
 19,
 16,
 16,
 16,
 19,
 16,
 13,
 19,
 19,
 16,
 16,
 16,
 13,
 16,
 16,
 16,
 16,
 15,
 16,
 16,
 16,
 20,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 16,
 19,
 16,
 16,
 16,
 20,
 16,
 16,
 20,
 16,
 16,
 16,
 16,
 11,
 11,
 16,
 18,
 15,
 17,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 1,
 16,
 16,
 20,
 16,
 16,
 19,
 19,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 20,
 20,
 16,
 15,
 16,
 16,
 16,
 16,
 16,
 1,
 16,
 16,
 16,
 16,
 16,
 16,

In [30]:
#Calculate the accuracy
#getting the test label
test_label=pd.read_csv('/Users/yongxingnie/Desktop/data science methods and tools/assignment4/homework4/data/test_label.csv', names=['t'])
test_label=test_label['t'].tolist()

total=len(test_label)

value=0
for i,j in zip(pred_2, test_label):
    if i==j:
        value+=1
    else:
        pass

print("Accuracy:\t", (value/total)*100, "%")

Accuracy:	 17.948034643570953 %
